# Essential basic functionality

In [2]:
import polars as pl
import numpy as np
from helper.jupyter import row

In [3]:
index = pl.date_range(pl.date(2000, 1, 1), pl.date(2000, 1, 8), eager=True).alias('index')
s = pl.DataFrame(dict(
    index=['a', 'b', 'c', 'd', 'e'],
    value=np.random.randn(5)
))

df = pl.DataFrame(np.random.randn(8, 3), schema=['A', 'B', 'C']).insert_column(0, index)

data = '''
       a         b         c
-0.173215  0.119209 -1.044236
-0.861849 -2.104569 -0.494929
+1.071804  0.721555 -0.706771
-1.039575  0.271860 -0.424972
+0.567020  0.276232 -1.087401
-0.673690  0.113648 -1.478427
+0.524988  0.404705  0.577046
-1.715002 -1.039268 -0.370647
'''

from helper.polars import to_dataframe
df = to_dataframe(data).insert_column(0, index)

## Head and tail

In [4]:
long_series = pl.Series(np.random.randn(1000))
long_series.head(5)

""
f64
0.70966
1.159269
0.743978
-0.591471
-0.350231


In [5]:
long_series.tail(3)

""
f64
0.298845
0.069854
-0.36121


## Attributes and underlying data

In [6]:
df[:2]

index,a,b,c
date,f64,f64,f64
2000-01-01,-0.173215,0.119209,-1.044236
2000-01-02,-0.861849,-2.104569,-0.494929


In [7]:
df = df.rename(str.lower)
df

index,a,b,c
date,f64,f64,f64
2000-01-01,-0.173215,0.119209,-1.044236
2000-01-02,-0.861849,-2.104569,-0.494929
2000-01-03,1.071804,0.721555,-0.706771
2000-01-04,-1.039575,0.27186,-0.424972
2000-01-05,0.56702,0.276232,-1.087401
2000-01-06,-0.67369,0.113648,-1.478427
2000-01-07,0.524988,0.404705,0.577046
2000-01-08,-1.715002,-1.039268,-0.370647


In [8]:
s['value'].to_arrow()

[
  -0.43140459430289096,
  -0.29538244153077464,
  0.7926928798463687,
  0.08436169656196303,
  1.399042528420122
]

In [9]:
s['index'].to_arrow()

[
  "a",
  "b",
  "c",
  "d",
  "e"
]

In [10]:
s['value'].to_numpy()

array([-0.43140459, -0.29538244,  0.79269288,  0.0843617 ,  1.39904253])

In [11]:
np.asarray(s['value'])

array([-0.43140459, -0.29538244,  0.79269288,  0.0843617 ,  1.39904253])

In [12]:
ser = (
pl.date_range(pl.date(2000, 1, 1), pl.date(2000, 1, 2), eager=True)
  .cast(pl.Datetime)
  .dt.replace_time_zone('CET')
)
ser.to_numpy()

array(['1999-12-31T23:00:00.000000', '2000-01-01T23:00:00.000000'],
      dtype='datetime64[us]')

In [13]:
# df.to_numpy()
df.select(pl.exclude('index')).to_numpy()

array([[-0.173215,  0.119209, -1.044236],
       [-0.861849, -2.104569, -0.494929],
       [ 1.071804,  0.721555, -0.706771],
       [-1.039575,  0.27186 , -0.424972],
       [ 0.56702 ,  0.276232, -1.087401],
       [-0.67369 ,  0.113648, -1.478427],
       [ 0.524988,  0.404705,  0.577046],
       [-1.715002, -1.039268, -0.370647]])

## Accelerated operations

## Flexible binary operations

In [14]:
data = '''
index   one       two     three
a  1.394981  1.772517       NaN
b  0.343054  1.912123 -0.050390
c  0.695246  1.478369  1.227435
d       NaN  0.279344 -0.613172
'''
df = to_dataframe(data)
df

index,one,two,three
str,f64,f64,f64
"""a""",1.394981,1.772517,NaN
"""b""",0.343054,1.912123,-0.05039
"""c""",0.695246,1.478369,1.227435
"""d""",NaN,0.279344,-0.613172


In [15]:
row_data = df.row(1, named=True)
column = df.select('index', value='two')
print(row_data)
print(column)

{'index': 'b', 'one': 0.343054, 'two': 1.912123, 'three': -0.05039}
shape: (4, 2)
┌───────┬──────────┐
│ index ┆ value    │
│ ---   ┆ ---      │
│ str   ┆ f64      │
╞═══════╪══════════╡
│ a     ┆ 1.772517 │
│ b     ┆ 1.912123 │
│ c     ┆ 1.478369 │
│ d     ┆ 0.279344 │
└───────┴──────────┘


In [16]:
# df.sub(row, axis="columns")
df.select(
    'index',
    *[pl.col(name) - val for name, val in row_data.items() if name != "index"]
)

index,one,two,three
str,f64,f64,f64
"""a""",1.051927,-0.139606,NaN
"""b""",0.0,0.0,0.0
"""c""",0.352192,-0.433754,1.277825
"""d""",NaN,-1.632779,-0.562782


In [17]:
# df.sub(column, axis="index")
df.select(
    'index',
    pl.exclude('index') - column['value']
)

index,one,two,three
str,f64,f64,f64
"""a""",-0.377536,0.0,NaN
"""b""",-1.569069,0.0,-1.962513
"""c""",-0.783123,0.0,-0.250934
"""d""",NaN,0.0,-0.892516


In [18]:
mi = pl.DataFrame(
    [(1, "a"), (1, "b"), (1, "c"), (2, "a")], 
    schema=["first", "second"],
    orient="row"
)
dfmi = pl.concat([mi, df.select(pl.exclude('index'))], how='horizontal')
dfmi

first,second,one,two,three
i64,str,f64,f64,f64
1,"""a""",1.394981,1.772517,NaN
1,"""b""",0.343054,1.912123,-0.05039
1,"""c""",0.695246,1.478369,1.227435
2,"""a""",NaN,0.279344,-0.613172


In [19]:
# dfmi.sub(column, axis=0, level="second")
new_column = column.join(dfmi.select('second'), left_on='index', right_on='second')
dfmi.select(
    pl.col('first', 'second'),
    pl.exclude('first', 'second') - new_column['value']
)

first,second,one,two,three
i64,str,f64,f64,f64
1,"""a""",-0.377536,0.0,NaN
1,"""b""",-1.569069,0.0,-1.962513
1,"""c""",-0.783123,0.0,-0.250934
2,"""a""",NaN,-1.493173,-2.385689


In [20]:
s = pl.Series(np.arange(10))
div, rem = s // 3, s % 3
row(s, div, rem)

""
i32
0
1
2
3
4
5
6
7
8


### Missing data / operations with fill values

In [21]:
# df2.loc["a", "three"] = 1.0
df2 = df.with_columns(
    pl.when(pl.col('index') == 'a')
      .then(1.0)
      .otherwise('three')
      .name.keep()
)
row(df, df2)

index,one,two,three
str,f64,f64,f64
"""a""",1.394981,1.772517,NaN
"""b""",0.343054,1.912123,-0.05039
"""c""",0.695246,1.478369,1.227435
"""d""",NaN,0.279344,-0.613172
index,one,two,three
str,f64,f64,f64
"""a""",1.394981,1.772517,1.0
"""b""",0.343054,1.912123,-0.05039
"""c""",0.695246,1.478369,1.227435


In [22]:
# df2.loc["a", "three"] = 1.0
df2 = df.update(pl.select(index=pl.lit('a'), three=1), on='index')

In [23]:
# df + df2
from helper.polars import align_op
align_op(df, df2, on='index', op=pl.Expr.add)

index,one,two,three
str,f64,f64,f64
"""a""",2.789962,3.545034,NaN
"""b""",0.686108,3.824246,-0.10078
"""c""",1.390492,2.956738,2.45487
"""d""",NaN,0.558688,-1.226344


### Flexible comparisons

In [24]:
# df.gt(df2)
align_op(df, df2, on='index', op=pl.Expr.gt)

index,one,two,three
str,bool,bool,bool
"""a""",false,false,true
"""b""",false,false,false
"""c""",false,false,false
"""d""",false,false,false


In [25]:
# df2.ne(df)
align_op(df2, df, on='index', op=pl.Expr.ne)

index,one,two,three
str,bool,bool,bool
"""a""",false,false,true
"""b""",false,false,false
"""c""",false,false,false
"""d""",false,false,false


### Boolean reductions

In [26]:
# (df > 0).all()
df.select((pl.exclude('index') > 0).all())

one,two,three
bool,bool,bool
true,true,false


In [27]:
# (df > 0).any()
df.select((pl.exclude('index') > 0).any())

one,two,three
bool,bool,bool
true,true,true


In [28]:
# (df > 0).any().any()
df.select(pl.any_horizontal(pl.exclude('index') > 0).any()).item()

True

In [29]:
df.is_empty()

False

In [30]:
pl.DataFrame(schema=list('ABC')).is_empty()

True

### Comparing if objects are equivalent

In [31]:
# df + df == df * 2
align_op(
    align_op(df, df, op=pl.Expr.add),
    df.select(
        'index',
        pl.exclude('index') * 2
    ),
    op=pl.Expr.eq
)

index,one,two,three
str,bool,bool,bool
"""a""",true,true,true
"""b""",true,true,true
"""c""",true,true,true
"""d""",true,true,true


In polars, NaN is equal to NaN

In [32]:
pl.Series([np.nan]) == pl.Series([np.nan])

""
bool
true


In [33]:
# (df + df).equals(df * 2)
align_op(df, df, op=pl.Expr.add).equals(
    df.select(
        'index',
        pl.exclude('index') * 2
    ),
)

True

In [34]:
df1 = pl.DataFrame({"index":[0, 1, 2], "col": [1.0, 0, np.nan]})
df2 = pl.DataFrame({"index":[2, 1, 0], "col": [np.nan, 0, 1.0]})
df1.equals(df2)

False

In [35]:
df1.equals(df2.sort('index'))

True

### Comparing array-like objects

In [36]:
pl.Series(["foo", "bar", "baz"]) == "foo"

""
bool
true
false
false


In [37]:
row(
pl.Series(["foo", "bar", "baz"]) == np.array(["foo", "bar", "qux"]),    
pl.Series(["foo", "bar", "baz"]) == pl.Series(np.array(["foo", "bar", "qux"]))
)

""
bool
false
false
false
""
bool
true
true
false


### Combining overlapping data sets

In [38]:
df1 = pl.DataFrame({
    "A": [1.0, np.nan, 3.0, 5.0, np.nan], 
    "B": [np.nan, 2.0, 3.0, np.nan, 6.0]
})

df2 = pl.DataFrame({
    "A": [5.0, 2.0, 4.0, np.nan, 3.0, 7.0],
    "B": [np.nan, np.nan, 3.0, 4.0, 6.0, 8.0],
})

row(df1, df2)

A,B
f64,f64
1.0,NaN
NaN,2.0
3.0,3.0
5.0,NaN
NaN,6.0
A,B
f64,f64
5.0,NaN
2.0,NaN


In [39]:
# df1.combine_first(df2)
(
df1
.with_row_index()
.join(
    df2.with_row_index(), on='index', how='left'
)
.fill_nan(None)
.select(
    [pl.coalesce(pl.col(c), pl.col(f'{c}_right')) for c in df1.columns]
)
)

A,B
f64,f64
1.0,null
2.0,2.0
3.0,3.0
5.0,4.0
3.0,6.0


### General DataFrame combine

In [40]:
# df1.combine(df2, combiner)

def combiner(x, y):
    return pl.when(x.fill_nan(None).is_null()).then(y).otherwise(x)
     
align_op(df1.with_row_index(), df2.with_row_index(), combiner, how='full', fill_value=None)

index,A,B
u32,f64,f64
0,1.0,NaN
1,2.0,2.0
2,3.0,3.0
3,5.0,4.0
4,3.0,6.0
5,7.0,8.0


## Descriptive statistics

In [41]:
df

index,one,two,three
str,f64,f64,f64
"""a""",1.394981,1.772517,NaN
"""b""",0.343054,1.912123,-0.05039
"""c""",0.695246,1.478369,1.227435
"""d""",NaN,0.279344,-0.613172


In [42]:
# df.mean(0)
df.select(pl.exclude('index').fill_nan(None).mean())

one,two,three
f64,f64,f64
0.811094,1.360588,0.187958


In [43]:
# df.mean(1)
df.select(
    'index',
    pl.mean_horizontal(pl.exclude('index').fill_nan(None))
)

index,one
str,f64
"""a""",1.583749
"""b""",0.734929
"""c""",1.133683
"""d""",-0.166914


In [44]:
# df.sum(0, skipna=False)
df.select(pl.exclude('index').sum())

one,two,three
f64,f64,f64
NaN,5.442353,NaN


In [45]:
# df.sum(axis=1, skipna=True)
df.select(
    'index',
    pl.sum_horizontal(pl.exclude('index').fill_nan(None))
)

index,one
str,f64
"""a""",3.167498
"""b""",2.204787
"""c""",3.40105
"""d""",-0.333828


In [49]:
cols = pl.exclude('index').fill_nan(None)
ts_stand = df.select(
    (cols - cols.mean()) / cols.std()
)
ts_stand.std()

one,two,three
f64,f64,f64
1.0,1.0,1.0


In [65]:
# xs_stand = df.sub(df.mean(1), axis=0).div(df.std(1), axis=0)
cols = pl.exclude('index').fill_nan(None)
cols_list = pl.concat_list(cols).list
xs_stand = df.select(
    'index',
    (cols - cols_list.mean()) / cols_list.std()
)
xs_stand.select(
    'index',
    cols_list.std()
)

index,one
str,f64
"""a""",1.0
"""b""",1.0
"""c""",1.0
"""d""",1.0


In [67]:
# df.cumsum()
cols = pl.exclude('index').fill_nan(None)
df.select(
    'index',
    cols.cum_sum()
)

index,one,two,three
str,f64,f64,f64
"""a""",1.394981,1.772517,null
"""b""",1.738035,3.68464,-0.05039
"""c""",2.433281,5.163009,1.177045
"""d""",null,5.442353,0.563873


In [73]:
np.mean(df['one'].to_numpy())

nan

In [79]:
series = pl.Series(np.random.randn(500))
series[np.arange(20, 500)] = np.nan
series[np.arange(10, 20)] = 5
series.n_unique()

12

In [82]:
series.drop_nans().n_unique()

11

### Summarizing data: describe

In [86]:
series = pl.Series(np.random.randn(1000))
series[np.arange(0, len(series), 2)] = None
series.describe()

statistic,value
str,f64
"""count""",500.0
"""null_count""",500.0
"""mean""",-0.00092
"""std""",0.999375
"""min""",-3.037615
"""25%""",-0.676407
"""50%""",0.011617
"""75%""",0.714109
"""max""",2.909347


In [91]:
frame = pl.DataFrame(np.random.randn(1000, 5), schema=["a", "b", "c", "d", "e"])

In [92]:
# frame.iloc[::2] = np.nan

frame = frame.select(
    pl.when(pl.int_range(0, pl.len()) % 2 == 0)
    .then(None)
    .otherwise(pl.all())
    .name.keep()
)

In [93]:
frame.describe()

statistic,a,b,c,d,e
str,f64,f64,f64,f64,f64
"""count""",500.0,500.0,500.0,500.0,500.0
"""null_count""",500.0,500.0,500.0,500.0,500.0
"""mean""",0.056895,-0.002257,-0.006452,-0.008162,0.157802
"""std""",1.008667,1.030909,0.989932,1.001297,1.036786
"""min""",-2.856179,-3.097654,-3.082604,-2.869447,-3.305509
"""25%""",-0.652431,-0.73133,-0.667962,-0.70158,-0.545181
"""50%""",0.028372,0.004722,-0.027605,0.011449,0.097324
"""75%""",0.682857,0.677287,0.651967,0.663524,0.830447
"""max""",2.678423,3.426341,3.229446,3.535054,3.418267


In [94]:
series.describe(percentiles=[0.05, 0.25, 0.75, 0.95])

statistic,value
str,f64
"""count""",500.0
"""null_count""",500.0
"""mean""",-0.00092
"""std""",0.999375
"""min""",-3.037615
"""5%""",-1.711111
"""25%""",-0.676407
"""75%""",0.714109
"""95%""",1.569069


In [95]:
s = pl.Series(["a", "a", "b", "b", "a", "a", None, "c", "d", "a"])
s.describe()

statistic,value
str,str
"""count""","""9"""
"""null_count""","""1"""
"""min""","""a"""
"""max""","""d"""


In [96]:
frame = pl.DataFrame({"a": ["Yes", "Yes", "No", "No"], "b": range(4)})
frame.describe()

statistic,a,b
str,str,f64
"""count""","""4""",4.0
"""null_count""","""0""",0.0
"""mean""",null,1.5
"""std""",null,1.290994
"""min""","""No""",0.0
"""25%""",null,1.0
"""50%""",null,2.0
"""75%""",null,2.0
"""max""","""Yes""",3.0


### Index of min/max values